In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import time
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import pickle
import os
import ultralytics
from ultralytics import YOLO
import cv2

In [58]:
labels = pd.read_csv('./datasets/traffic-signs-preprocessed/label_names.csv')
with open('./datasets/traffic-signs-preprocessed/mean_image_rgb.pickle', 'rb') as f:
    mean = pickle.load(f, encoding='latin1')  # dictionary type

In [3]:
print(labels.head())
print()

   ClassId              SignName
0        0  Speed limit (20km/h)
1        1  Speed limit (30km/h)
2        2  Speed limit (50km/h)
3        3  Speed limit (60km/h)
4        4  Speed limit (70km/h)



# Loading models
## YOLO

In [38]:

path_to_cfg = './YOLOV5/yolov5,runs/train/exp/weights'
path_to_weights = './YOLOV5/yolov5/runs/train/exp2/weights/best.onnx'
w = "./best.onnx"

#network = cv2.dnn.readNetFromDarknet(path_to_cfg, path_to_weights)
network = cv2.dnn.readNetFromONNX(path_to_weights)
# To use with GPU
network.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
network.setPreferableTarget(cv2.dnn.DNN_TARGET_OPENCL_FP16)

In [39]:
layers_all = network.getLayerNames()

# Check point
#print(layers_all)

# Getting only detection YOLO v3 layers that are 82, 94 and 106
layers_names_output = [layers_all[i - 1] for i in network.getUnconnectedOutLayers()]

# Check point
print()
#print(layers_names_output)  # ['yolo_82', 'yolo_94', 'yolo_106']

# Pytorch

In [66]:
import torchvision
import torch
PATH = "./tl_detection.pth"
model = torchvision.models.resnet34(weights = torchvision.models.ResNet34_Weights.IMAGENET1K_V1)
model.fc = torch.nn.Sequential(
    model.fc,
    torch.nn.ReLU(),
    torch.nn.Linear(1000, 512, True),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 43, True),
    torch.nn.Softmax(dim=1)
)
            
model.load_state_dict(torch.load(PATH))
model.eval();

In [67]:
# Minimum probability to eliminate weak detections
probability_minimum = 0.2

# Setting threshold to filtering weak bounding boxes by non-maximum suppression
threshold = 0.2

# Generating colours for bounding boxes
# randint(low, high=None, size=None, dtype='l')
colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

# Check point
print(type(colours))  # <class 'numpy.ndarray'>
print(colours.shape)  # (43, 3)
print(colours[0])  # [25  65 200]

<class 'numpy.ndarray'>
(43, 3)
[ 17 142  97]


In [179]:
img = cv2.imread("./stop.jpeg")
#img = img.astype(np.float32) / 255.0  # normalize to [0, 1]
#resized_image = cv2.resize(img, (640, 640))
img2 = img.copy()
img2 = np.array(img2)
h, w = img.shape[:2]
h,w

(219, 230)

In [180]:
import torchvision.transforms as transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), 
     transforms.Resize((224, 224), antialias=True)
    
    ])

In [181]:
blob = cv2.dnn.blobFromImage(img,1,(640, 640),[ 0,0,0], crop=False, swapRB=True)
network.setInput(blob)
output_from_network = network.forward()


In [182]:
output_from_network
bounding_boxes = []
confidences = []
class_numbers = []



In [183]:
# Going through all output layers after feed forward pass
for result in output_from_network:
    # Going through all detections from current output layer
    for detected_objects in result:
        # Getting 80 classes' probabilities for current detected object
        scores = detected_objects[5:]
        # Getting index of the class with the maximum value of probability
        class_current = np.argmax(scores)
        # Getting value of probability for defined class
        confidence_current = scores[class_current]

        # Eliminating weak predictions by minimum probability
        if confidence_current > probability_minimum:
            # Scaling bounding box coordinates to the initial frame size
            box_current = detected_objects[0:4] * np.array([w, h, w, h])
            #print(box_current)
            # Getting top left corner coordinates
            x_center, y_center, box_width, box_height = box_current
            x_min = int(x_center - (box_width / 2))
            y_min = int(y_center - (box_height / 2))

            # Adding results into prepared lists
            bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
            confidences.append(float(confidence_current))
            class_numbers.append(class_current)
                

# Implementing non-maximum suppression of given bounding boxes
results = cv2.dnn.NMSBoxes(bounding_boxes, confidences, probability_minimum, threshold)
#print(results)
# Checking if there is any detected object been left
if len(results) > 0:
    # Going through indexes of results
    for i in results.flatten():
        # Bounding box coordinates, its width and height
        x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
        box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]
            
            
        # Cut fragment with Traffic Sign
        c_ts = img[y_min:y_min+int(box_height), x_min:x_min+int(box_width), :]
        # print(c_ts.shape)
            
        if c_ts.shape[:1] == (0,) or c_ts.shape[1:2] == (0,):
            pass
        else:
            # Getting preprocessed blob with Traffic Sign of needed shape
            blob_ts = cv2.dnn.blobFromImage(c_ts, 1 / 255.0, size=(32, 32), swapRB=True, crop=False)
            blob_ts[0] = blob_ts[0, :, :, :] #- mean['mean_image_rgb']
            blob_ts = blob_ts.transpose(0, 2, 3, 1)
            # plt.imshow(blob_ts[0, :, :, :])
            #plt.show()
            blob_ts = np.squeeze(blob_ts)
            #print(blob_ts.shape)
            blob_ts = transform(blob_ts)
            #print(blob_ts.shape)
           
            # Feeding to the Keras CNN model to get predicted label among 43 classes
            scores = model(blob_ts.unsqueeze(0))

            # Scores is given for image with 43 numbers of predictions for each class
            # Getting only one class with maximum value
            prediction = torch.argmax(scores).cpu().numpy()
            print(labels['SignName'][prediction])
            print(prediction)
            
            # Colour for current bounding box
            colour_box_current = colours[class_numbers[i]].tolist()
            print(colour_box_current)
            # Green BGR
            #colour_box_current = [0, 255, 61]
            
            # Yellow BGR
#             colour_box_current = [0, 255, 255]
            print(img2.shape)
            #print(img2[:,::-1].shape)
            #print(img2)
            # Drawing bounding box on the original current frame
            cv2.rectangle(img2, (x_min, y_min),
                              (x_min + box_width, y_min + box_height),
                              (0,0,0),  thickness=2)
            
            print((x_min, y_min),
                              (x_min + box_width, y_min + box_height))
            
# Saving image
cv2.imwrite('result.png', img2)

Stop
14
[94, 205, 35]
(219, 230, 3)
(-669, -6505) (9959, 14658)
Stop
14
[94, 205, 35]
(219, 230, 3)
(-10829, -31737) (33428, 55672)


True